In [38]:
import os
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib_venn import venn2

In [39]:
file_path = 'data\\val.json'

IS_TRAIN = False

res_path = 'val'

In [41]:
if not os.path.exists(f'data\\{res_path}'):
    os.makedirs(f'data\\{res_path}')

In [42]:
# Открываем файл и загружаем данные
with open(file_path, 'r') as file:
    data = json.load(file)

In [43]:
df = pd.DataFrame(data.values(), data.keys())
df = df.reset_index()
df = df.rename(columns = {'index': 'user_id'})
df

,user_id,target,features
0,user_127756,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."
1,user_127757,male,"{'orders': [{'site-id': 16, 'orders': [{'creat..."
2,user_127758,female,"{'orders': [{'site-id': 149, 'orders': [{'crea..."
3,user_127759,female,"{'orders': [{'site-id': 93, 'orders': [{'creat..."
4,user_127760,female,"{'visits': [{'site-id': 3, 'first-seen': 16836..."
...,...,...,...
27442,user_155198,female,"{'orders': [{'site-id': 7, 'orders': [{'create..."
27443,user_155199,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
27444,user_155200,female,"{'orders': [{'site-id': 34, 'orders': [{'creat..."
27445,user_155201,male,"{'orders': [{'site-id': 218, 'orders': [{'crea..."


In [44]:
def process_dataframe(df):
    # Добавляем столбец 'length', содержащий количество элементов в словаре
    df['length'] = df['features'].apply(lambda x: sum(len(v) if isinstance(v, list) else 1 for v in x.values()))

    # Преобразуем столбец 'features' с помощью json_normalize
    df_filtered = df.join(pd.json_normalize(df['features'])).drop('features', axis='columns')

    return df_filtered

def explode_column(df, level_labels):
    for level in level_labels:
        df = df.explode(level, ignore_index=True)
        df = df.rename(columns={level: level+'_old'})
        df = df.join(pd.json_normalize(df[level+'_old'])).drop(level+'_old', axis='columns')
    return df

def remove_outliers(df):
    Q1 = df['length'].quantile(0.25)
    Q3 = df['length'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.2 * IQR
    upper_bound = Q3 + 1.2 * IQR
    return df[(df['length'] <= upper_bound)]

def with_outliers(df):
    Q1 = df['length'].quantile(0.25)
    Q3 = df['length'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.2 * IQR
    upper_bound = Q3 + 1.2 * IQR
    return df[(df['length'] > upper_bound)]

In [45]:
# Применяем первую часть предобработки к исходному DataFrame
df_filtered = process_dataframe(df)
df_filtered

,user_id,target,length,orders,visits,site-meta,exchange-sessions,last-visits-in-categories
0,user_127756,female,8,"[{'site-id': 407, 'orders': [{'created-at': 16...","[{'site-id': 3, 'first-seen': 1697125590, 'las...",[{'site-id': 407}],"[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,16,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'site-id': 3, 'first-seen': 1698423484, 'las...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,50,"[{'site-id': 149, 'orders': [{'created-at': 16...","[{'site-id': 391, 'first-seen': 1685615371, 'l...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...",NaN,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,34,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'site-id': 495, 'first-seen': 1687325784, 'l...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,12,NaN,"[{'site-id': 3, 'first-seen': 1683626530, 'las...","[{'site-id': 46}, {'site-id': 29}, {'site-id':...",NaN,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...,...,...,...,...,...
27442,user_155198,female,22,"[{'site-id': 7, 'orders': [{'created-at': 1625...","[{'site-id': 3, 'first-seen': 1688903751, 'las...","[{'site-id': 49, 'recency': 1, 'frequency': 1,...","[{'landed-at': 1650900543, 'sites': [71, 326, ...","[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,7,"[{'site-id': 21, 'orders': [{'created-at': 169...","[{'site-id': 21, 'first-seen': 1697887543, 'la...","[{'site-id': 21}, {'site-id': 17}]",NaN,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,23,"[{'site-id': 34, 'orders': [{'created-at': 158...","[{'site-id': 22, 'first-seen': 1686412090, 'la...","[{'site-id': 34, 'recency': 1, 'frequency': 1,...",NaN,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,27,"[{'site-id': 218, 'orders': [{'created-at': 16...","[{'site-id': 555, 'first-seen': 1696063525, 'l...","[{'site-id': 218, 'recency': 1, 'frequency': 1...",NaN,"[{'category': 'other', 'last-visit-at': 169911..."


In [46]:
if IS_TRAIN:
    # Удаляем выбросы по 'length' для каждой группы в 'target'
    df_filtered_out = df_filtered.groupby('target', group_keys=False).apply(remove_outliers)
    df_filtered_out = df_filtered_out.reset_index(drop=True)
else:
    df_filtered_out = df_filtered.copy()
df_filtered_out

,user_id,target,length,orders,visits,site-meta,exchange-sessions,last-visits-in-categories
0,user_127756,female,8,"[{'site-id': 407, 'orders': [{'created-at': 16...","[{'site-id': 3, 'first-seen': 1697125590, 'las...",[{'site-id': 407}],"[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,16,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'site-id': 3, 'first-seen': 1698423484, 'las...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,50,"[{'site-id': 149, 'orders': [{'created-at': 16...","[{'site-id': 391, 'first-seen': 1685615371, 'l...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...",NaN,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,34,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'site-id': 495, 'first-seen': 1687325784, 'l...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,12,NaN,"[{'site-id': 3, 'first-seen': 1683626530, 'las...","[{'site-id': 46}, {'site-id': 29}, {'site-id':...",NaN,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...,...,...,...,...,...
27442,user_155198,female,22,"[{'site-id': 7, 'orders': [{'created-at': 1625...","[{'site-id': 3, 'first-seen': 1688903751, 'las...","[{'site-id': 49, 'recency': 1, 'frequency': 1,...","[{'landed-at': 1650900543, 'sites': [71, 326, ...","[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,7,"[{'site-id': 21, 'orders': [{'created-at': 169...","[{'site-id': 21, 'first-seen': 1697887543, 'la...","[{'site-id': 21}, {'site-id': 17}]",NaN,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,23,"[{'site-id': 34, 'orders': [{'created-at': 158...","[{'site-id': 22, 'first-seen': 1686412090, 'la...","[{'site-id': 34, 'recency': 1, 'frequency': 1,...",NaN,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,27,"[{'site-id': 218, 'orders': [{'created-at': 16...","[{'site-id': 555, 'first-seen': 1696063525, 'l...","[{'site-id': 218, 'recency': 1, 'frequency': 1...",NaN,"[{'category': 'other', 'last-visit-at': 169911..."


### Orders

In [47]:
# Выбираем нужные столбцы для df_orders
df_orders = df_filtered_out[['user_id', 'target', 'orders', 'length']].copy()

In [48]:
# Задаем метки уровней для функции explode_column
levels_labels = ['orders', 'orders', 'items']

# Раскрываем все признаки в orders
df_orders_flat = explode_column(df_orders, levels_labels)

# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
df_orders_flat = df_orders_flat.reset_index(drop=True)
df_orders_flat

,user_id,target,length,site-id,created-at,id,count,general-category-path,brand-id
0,user_127756,female,8,407.0,1.697126e+09,item_676374,NaN,NaN,NaN
1,user_127757,male,16,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
2,user_127757,male,16,16.0,1.637513e+09,item_5403882,2.0,"[90578, 90574, 198118]",1237.0
3,user_127757,male,16,16.0,1.637948e+09,item_1627033,1.0,"[91498, 91497, 91461, 198119]",1.0
4,user_127757,male,16,16.0,1.638631e+09,item_554781,1.0,"[90629, 90625, 10599873, 198119]",3519.0
...,...,...,...,...,...,...,...,...,...
815454,user_155202,female,45,53.0,1.684604e+09,item_650360,1.0,NaN,NaN
815455,user_155202,female,45,53.0,1.684604e+09,item_650361,1.0,NaN,NaN
815456,user_155202,female,45,54.0,1.420435e+09,NaN,NaN,NaN,NaN
815457,user_155202,female,45,300.0,1.688068e+09,item_650362,1.0,"[818863, 15068776, 7811881, 7877999]",915491.0


In [49]:
df_orders_flat.to_parquet(f'data\\{res_path}\\df_orders_flat.parquet')

### Visits

In [50]:
# Выбираем нужные столбцы для df_visits
df_visits = df_filtered_out[['user_id', 'target', 'visits', 'length']].copy()

In [51]:
# Задаем метки уровней для функции explode_column
levels_labels = ['visits', 'visits']

# Раскрываем все признаки в orders
df_visits_flat = explode_column(df_visits, levels_labels)

# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
df_visits_flat = df_visits_flat.reset_index(drop=True)
df_visits_flat

,user_id,target,length,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
1,user_127756,female,8,3.0,1.697126e+09,1.697133e+09,1.697133e+09,0.0,1.0,NaN,NaN,NaN
2,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698855e+09,626.0,3.0,NaN,NaN,NaN
3,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698931e+09,0.0,1.0,NaN,NaN,NaN
4,user_127756,female,8,407.0,1.697125e+09,1.699115e+09,1.698947e+09,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
561073,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697236e+09,581.0,3.0,NaN,NaN,NaN
561074,user_155201,male,27,13.0,1.691565e+09,1.697718e+09,1.697718e+09,0.0,1.0,NaN,NaN,NaN
561075,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696679e+09,0.0,1.0,NaN,NaN,NaN
561076,user_155202,female,45,3.0,1.696679e+09,1.696864e+09,1.696680e+09,0.0,1.0,NaN,NaN,NaN


In [52]:
df_visits_flat.to_parquet(f'data\\{res_path}\\df_visits_flat.parquet')

### Last-visits-in-categories

In [53]:
# Выбираем нужные столбцы для df_last_visits_cat
df_last_visits_cat = df_filtered_out[['user_id', 'target', 'last-visits-in-categories', 'length']].copy()

In [54]:
# Задаем метки уровней для функции explode_column
levels_labels = ['last-visits-in-categories']

# Раскрываем все признаки в orders
df_last_visits_cat_flat = explode_column(df_last_visits_cat, levels_labels)

# Сбрасываем индексы, чтобы получить DataFrame аналогичный исходному
df_last_visits_cat_flat = df_last_visits_cat_flat.reset_index(drop=True)
df_last_visits_cat_flat

,user_id,target,length,category,last-visit-at
0,user_127756,female,8,other,1.697133e+09
1,user_127756,female,8,education,1.699115e+09
2,user_127757,male,16,electronics,1.698422e+09
3,user_127757,male,16,other,1.698423e+09
4,user_127757,male,16,shoes,1.698424e+09
...,...,...,...,...,...
156326,user_155201,male,27,education,1.695401e+09
156327,user_155201,male,27,luxury,1.697236e+09
156328,user_155201,male,27,entertainment_services,1.697235e+09
156329,user_155201,male,27,media,1.697718e+09


In [55]:
df_last_visits_cat_flat.to_parquet(f'data\\{res_path}\\df_last_visits_cat_flat.parquet')